In [2]:
import mapclassify
import geopandas as gpd
import pandas as pd
from pandas.api.types import is_numeric_dtype
import re


class MapClassify:
    def __init__(self,file,method,column,K_classes=None,bins=None):
        self.file=file
        self.K_classes=K_classes
        self.method=method
        self.column=column
        self.gdf=None
        self.bins=bins
        self.__read_file()
        if self.method not in mapclassify.CLASSIFIERS:
            raise ValueError('unkown method write right method')
    
    def __read_file(self):
        self.gdf=gpd.read_file(self.file)
    def classify(self):
        if is_numeric_dtype(self.gdf[self.column]) == False:
            raise ValueError('column must be numeric')
        try:
          param_map = {
              'HeadTailBreaks': {},
              "StdMean":{},
              'Percentiles': {'pct': self.bins},
              'UserDefined': {'bins': self.bins},
          }
        
          classifier_class = getattr(mapclassify, self.method)
    
          params = {'y': self.gdf[self.column]}
          if self.method in param_map:
              params.update(param_map[self.method])
          else:
              params['k'] = self.K_classes
      
          if self.method == 'HeadTailBreaks':
              return classifier_class(**params)
          return classifier_class(**params)
        except Exception as e:
          raise ValueError(f'error classifing data:{e} ')
    def prepare_data(self):
        try:
          classify=self.classify()
          filterd_gdf=self.gdf[['geometry']]
          ranges=[]
          for x in classify.get_legend_classes():
              
               low, high = map(float, re.findall(r"[-+]?\d*\.\d+|\d+", x))
               ranges.append([low,high])
          df_classes = pd.DataFrame({
          "class": range(len(classify.counts)),
          "count": classify.counts,
          "range": ranges,
        })


          filterd_gdf = filterd_gdf.merge(df_classes, left_index=True, right_on="class")
          
        #   filterd_gdf['range']=ranges
          
          return filterd_gdf
        except Exception as e:
            raise ValueError(f'cannot prepare data:{e}')

In [ ]:
from geo.Geoserver import Geoserver

In [1]:
import mapclassify
mapclassify.CLASSIFIERS

('BoxPlot',
 'EqualInterval',
 'FisherJenks',
 'FisherJenksSampled',
 'HeadTailBreaks',
 'JenksCaspall',
 'JenksCaspallForced',
 'JenksCaspallSampled',
 'MaxP',
 'MaximumBreaks',
 'NaturalBreaks',
 'Quantiles',
 'Percentiles',
 'PrettyBreaks',
 'StdMean',
 'UserDefined')

In [ ]:
aa=getattr(mapclassify,'UserDefined')

In [ ]:
asa=gpd.read_file('test_data.shp')

In [ ]:
import mapclassify
import geopandas as gpd
import pandas as pd
from pandas.api.types import is_numeric_dtype
import re

class MapClassify:
    """
    A utility class for classifying geospatial data using various mapclassify methods.

    This class reads a geospatial file, classifies a specified numeric column
    using a chosen mapclassify method, and prepares a GeoDataFrame with
    classification results, including class ranges and assigned colors.
    """

    def __init__(self, file, method, column, colors, K_classes=None, bins=None):
        """
        Initializes the MapClassify object.

        Args:
            file (str): The path to the geospatial file (e.g., shapefile, GeoJSON).
            method (str): The mapclassify method to use for classification.
                Must be one of the available mapclassify.CLASSIFIERS.
            column (str): The name of the numeric column to be classified.
            colors (list): A list of colors to assign to each class.
            K_classes (int, optional): The number of classes for classification methods that
                require it (e.g., EqualInterval, FisherJenks). Defaults to None.
            bins (list, optional): A list of custom bin edges for specific methods
                (e.g., UserDefined, Percentiles). Defaults to None.
        
        Raises:
            ValueError: If an unknown classification method is provided.
        """
        self.file = file
        self.K_classes = K_classes
        self.method = method
        self.column = column
        self.gdf = None
        self.bins = bins
        self.colors = colors
        self._read_file()
        if self.method not in mapclassify.CLASSIFIERS:
            raise ValueError('unknown method, please provide a valid mapclassify method.')

    def _read_file(self):
        """
        Reads the geospatial file into a GeoDataFrame.

        This is an internal method and should not be called directly.
        """
        self.gdf = gpd.read_file(self.file)

    def _classify(self):
        """
        Classifies the specified column using the selected mapclassify method.

        This is an internal method that handles the logic of calling the
        correct mapclassify classifier with the appropriate parameters.

        Raises:
            ValueError: If the specified column is not numeric.
            ValueError: If an error occurs during the classification process.

        Returns:
            mapclassify.MapClassifier: The classification object.
        """
        if not is_numeric_dtype(self.gdf[self.column]):
            raise ValueError('column must be numeric')
        try:
            param_map = {
                'HeadTailBreaks': {},
                'Percentiles': {'pct': self.bins},
                'UserDefined': {'bins': self.bins},
            }

            classifier_class = getattr(mapclassify, self.method)
            params = {'y': self.gdf[self.column]}
            if self.method in param_map:
                params.update(param_map[self.method])
            else:
                params['k'] = self.K_classes
            return classifier_class(**params)
        except Exception as e:
            raise ValueError(f'error classifying data: {e}')

    def prepare_data(self):
        """
        Prepares a GeoDataFrame with the classification results and colors.

        This method performs the classification, extracts the class ranges,
        and merges the classification results and colors back into a
        new GeoDataFrame.

        Raises:
            ValueError: If an error occurs during data preparation.

        Returns:
            gpd.GeoDataFrame: A GeoDataFrame containing the 'geometry',
                              'class', 'count', 'range', and 'color' columns.
        """
        try:
            classify = self._classify()
            filterd_gdf = self.gdf[['geometry']]
            ranges = []
            for x in classify.get_legend_classes():
                low, high = map(float, re.findall(r"[-+]?\d*\.\d+|\d+", x))
                ranges.append([low, high])
            
            df_classes = pd.DataFrame({
                "class": range(len(classify.counts)),
                "count": classify.counts,
                "range": ranges,
                "color": [self.colors[x] for x in range(len(ranges))]
            })
            
            filterd_gdf = filterd_gdf.merge(df_classes, left_index=True, right_on="class")
            return filterd_gdf
        except Exception as e:
            raise ValueError(f'cannot prepare data: {e}')



In [104]:
asa.columns.to_list()

['AREA',
 'PERIMETER',
 'CNTY_',
 'CNTY_ID',
 'NAME',
 'FIPS',
 'FIPSNO',
 'CRESS_ID',
 'BIR74',
 'SID74',
 'NWBIR74',
 'BIR79',
 'SID79',
 'NWBIR79',
 'SIDR74',
 'SIDR79',
 'NWR74',
 'NWR79',
 'geometry']

In [3]:
qq=MapClassify(file=r'C:\Users\Yousef-Ragab\Desktop\map classification\test_data.shp',K_classes=3,bins=[1,5,30,57],method='PrettyBreaks',column='SID79')

In [5]:
aa=qq.prepare_data()

In [6]:
aa

,geometry,class,count,range
0,"POLYGON ((-81.47276 36.23436, -81.54084 36.272...",0,99,"[0.0, 50.0]"
1,"POLYGON ((-81.23989 36.36536, -81.24069 36.379...",1,1,"[50.0, 100.0]"


In [ ]:
mapclassify.PrettyBreaks()

In [ ]:
qqa=mapclassify.StdMean(y=asa.SID79)

In [ ]:
qqa

In [ ]:
cc.range[0]

In [ ]:
import re

s = "[ 0.00,  8.36]"
low, high = map(float, re.findall(r"[-+]?\d*\.\d+|\d+", s))
print(low, high)


In [ ]:
q=[]
for x in cc.get_legend_classes():
    

     s = x
     low, high = map(float, re.findall(r"[-+]?\d*\.\d+|\d+", s))
     q.append([low,high])

In [ ]:
f=cc.get_legend_classes()[0]

In [ ]:
f.split(',')

In [ ]:
s=cc.get_legend_classes()
s = s.strip("[]")
low, high = map(float, s.split(","))
print(low, high)

In [ ]:
mapclassify.CLASSIFIERS

In [ ]:
from geoserver import Geoserver
import geopandas as gpd


In [ ]:
geoserver=Geoserver()

In [ ]:
layer=geoserver.get_vector_layer(layername='populated_places',workspace='ne')

In [ ]:
qq=MapClassify(file=layer,K_classes=4,method='Quantiles',column='pop_max')

In [ ]:
import geopandas
import libpysal
import matplotlib.pyplot as plt

import mapclassify

In [ ]:
libpysal.examples.explain("sids2")

In [ ]:
pth = libpysal.examples.get_path("sids2.shp")
gdf = geopandas.read_file(pth)
gdf.head()

In [ ]:
from geoserver import Geoserver

In [ ]:
geoserver=Geoserver()

In [ ]:
import tempfile
with tempfile.TemporaryDirectory()as tmp:
    gdf.to_file(f'{tmp}/map_classify.shp')
    zip_path=geoserver.create_zipfilepath(tmpdir=tmp)
    geoserver.zip_files(zip_file_path=zip_path,tempdir=tmp)
    upload=geoserver.upload_shapefile(workspace='cite',datastore='map_classify',zip_path=zip_path)

In [ ]:
upload

In [ ]:
rer=gpd.read_file(layer)

In [ ]:
asdsd=geoserver.upload_raster_data(workspace='sdf',coveragestore='dsfdf',layername='dsfdsf',file_path='ss.geojson')

In [ ]:
ww=qq.classify()

In [ ]:
ww.classes[0]

In [ ]:
es.pop_max.dtype

In [ ]:
aa=gpd.read_file(r'ss.shp')

In [ ]:
aa

In [ ]:
from shapely.geometry import MultiPolygon, Polygon

aa["geometry"] = aa["geometry"].apply(
    lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom
)


In [ ]:
aa.to_file('qq.geojson')

In [ ]:
from pandas.api.types import is_numeric_dtype

In [ ]:
'sss'.isnumeric()

In [ ]:
is_numeric_dtype(ss.Index)

In [ ]:
mapclassify.CLASSIFIERS

In [ ]:
gdf=gpd.read_file(r"C:\Users\Yousef-Ragab\Desktop\saudiArabia\metro-lines-in-riyadh-2024.geojson")

In [ ]:
gdf.assign(cl)

In [ ]:
import pyrosm

In [ ]:
import matplotlib

In [ ]:
data=pyrosm.OSM(filepath=r'C:\Users\Yousef-Ragab\Desktop\gaddah\planet_39.099_21.203_de9813ff.osm.pbf')

In [ ]:
es=data.get_landuse()

In [ ]:
es.plot()

In [ ]:
es

In [ ]:
qq=es[['geometry','military','id']]

In [ ]:
mapclassify.CLASSIFIERS

In [ ]:
qq.dissolve(by='military',aggfunc='min')

In [ ]:
clsify=mapclassify.EqualInterval(y=es['id'],k=4)

In [ ]:
import numpy as np


In [ ]:
wew=np.array([2,4,6,7,8,9])

In [ ]:
clsify=mapclassify.EqualInterval(y=wew,k=3)

In [ ]:
clsify

In [ ]:
clsify.get_legend_classes()

In [ ]:
es['class']=clsify.yb

In [ ]:
es.dissolve(by='class')

In [ ]:
clsify

In [ ]:
clsify

In [ ]:
es['class']=clsify.yb

In [ ]:
ww=es[['geometry','class']]

In [ ]:
ww.dissolve(by='class')

In [ ]:
clsify

In [ ]:
aa=clsify.classes

In [ ]:
aa